## Playground 3

Workshopping LLM Output Code Execution and Advanced RAG

In [1]:
import pandas as pd
import numpy as np
import json, os, pprint
import matplotlib.pyplot as plt
import plotly.express as px
import random
from langchain_openai import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.tools import tool
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain.output_parsers import JsonOutputToolsParser, JsonOutputKeyToolsParser
from langchain.agents import AgentExecutor, create_openai_tools_agent, create_react_agent, Tool
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_experimental.utilities import PythonREPL
from langchain import hub
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.callbacks import Callbacks
from langchain.prompts import ChatPromptTemplate

In [2]:
os.environ["OPENAI_API_KEY"] = ""

Creating a Python execution tool

In [16]:
python_repl = PythonREPL()

python_repl.run("import numpy as np; x=np.random.rand(3,2); print(x)")

'[[0.06119131 0.76473527]\n [0.82000326 0.26274933]\n [0.85977688 0.88628719]]\n'

In [3]:
python_repl = PythonREPL()

exec_python_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)

In [4]:
llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0.1, streaming=True)

In [29]:
llm_with_tools = llm.bind_tools([exec_python_tool])

python_create_chain = llm_with_tools | JsonOutputToolsParser()

In [30]:
d = python_create_chain.invoke("can you generate a random vector in python of size 3 by 2?")
d

[{'type': 'python_repl',
  'args': {'__arg1': 'import numpy as np\n\n# Generate a random vector of size 3x2\nrandom_vector = np.random.rand(3, 2)\n\nprint(random_vector)'}}]

In [33]:
d[0]["args"]["__arg1"]

'import numpy as np\n\n# Generate a random vector of size 3x2\nrandom_vector = np.random.rand(3, 2)\n\nprint(random_vector)'